In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
from IPython.display import display, HTML
from difflib import SequenceMatcher
import math
import Levenshtein
import shutil

import importlib.util
spec = importlib.util.spec_from_file_location("module.name","sanskrit_transcoder/transcoder.py")
transcoder = importlib.util.module_from_spec(spec)
spec.loader.exec_module(transcoder)
transcoder.transcoder_set_dir('sanskrit_transcoder/data/transcoder');

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)

In [3]:
def printlines(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            print(line)
            
    return 

def getline(fname,tag = 'input line : ') :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith(tag)) :
                l = line[len(tag):-1]
    return l

def getconvertedline(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("converted line : ")) :
                l = line[17:-1]
               
    return l

def getcovertedchar(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("converted line char :")) :
                l = line[21:-1].split(',')
               
    return l


def fetched_data(fname) :
    fd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return fd


def dependency_structure(fname):
    ds = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return ds


def inputline_(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("input line : ")):
                return line[13:-1]
            
            
def conflicts_dataframe(fname) :
    cd = pd.read_csv(fname,encoding = 'utf-8' ,sep = ',')
    return cd

def originalline_(fname) :
    with open(fname,encoding = 'utf-8' ,mode ='r') as fr :
        for line in fr.readlines() :
            if(line.startswith("SENT: ")):
                return line[6:-1]


In [14]:
data = pd.DataFrame(columns = ['id','fname','inputline','originalline','cword','pos','dep','hword','clemma','cmorph','cauxi_inf','cpre_verb','hlemma','hmorph','hauxi_inf','hpre_verb'])
data

,id,fname,inputline,originalline,cword,pos,dep,hword,clemma,cmorph,cauxi_inf,cpre_verb,hlemma,hmorph,hauxi_inf,hpre_verb


In [4]:
path = 'workdata/lines_submit/adding/phase3/'
fnames = os.listdir(path)
len(fnames)

2948

In [ ]:
dswrng = []
id_ = 0
for fname in fnames[0:1000]:
    fname1 = path+fname+'/dataframe_towork.csv'
    fname2 = path+fname+'/dependency.txt'
    fname3 = path+fname+'/input_line.txt'
    fname4 = path+fname+'/original_line.txt'
    
    ds = dependency_structure(fname2)
    fd = fetched_data(fname1)
    il = inputline_(fname3)
    ol = originalline_(fname4)
    dswo = ds.wordorder.unique()
    dswos = [int(k) for k in dswo]
    for j in ds.index :
        cword = ds.loc[j,'word']
        cwo = ds.loc[j,'wordorder']
        hwo = ds.loc[j,'head']
        if not str(hwo) == 'nan':
            hwo = re.sub('\*|\+,|\.0|[^0-9\.\+\*\_\-]','',str(hwo))
        dep = ds.loc[j,'dep']
        pos = ds.loc[j,'POS']
        f1 = fd.loc[fd['wordorder'] == cwo]
        
        if str(hwo) == 'nan' :
            
            for ci in f1.index :
                data.loc[id_] = [id_,fname,il,ol,cword,pos,dep,'',f1.loc[ci,'lemma'],f1.loc[ci,'morph'],f1.loc[ci,'aux_inf'],f1.loc[ci,'pre_verb'],
                            '','','','']
                id_+=1
        
        elif int(hwo) in dswos:
            hword = ds.loc[ds['wordorder']==int(hwo)].iloc[0]['word']
            f2 = fd.loc[fd['wordorder'] == int(hwo)]
            for ci in f1.index :
                for hi in f2.index :

                    data.loc[id_] = [id_,fname,il,ol,cword,pos,dep,hword,f1.loc[ci,'lemma'],f1.loc[ci,'morph'],f1.loc[ci,'aux_inf'],f1.loc[ci,'pre_verb'],
                                f2.loc[hi,'lemma'],f2.loc[hi,'morph'],f2.loc[hi,'aux_inf'],f2.loc[hi,'pre_verb']]
                    id_+=1
        else :
            print(hwo)
            if fname not in dswrng :
                dswrng.append(fname)
                
            

5
4
5
6
9
7
12
7
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
20_1
7
13
13
4
4
5
5
4
10
10
9
9
7
5
9
7
13
9
9
9


In [78]:
dswrng

['1004',
 '114',
 '1212',
 '1215',
 '1232',
 '1274',
 '1310',
 '143',
 '1449',
 '1468',
 '1484',
 '1486',
 '1619',
 '163',
 '1632']

In [75]:
fname


'1648'